**FidlTrack_predict** allows to infer and optimise trajectory fidelity of single-molecule recordings.

Sequentially run the following cells.

**Setup**

In [ ]:
#@markdown Launch this cell to initialise the notebook, you only need to run it once.
from google.colab import output
is_dark = output.eval_js('document.documentElement.matches("[theme=dark]")')

!pip install -q wget
!pip install -q scipy

import wget
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits import axes_grid1

base_url = "https://drive.usercontent.google.com/download?id={}&authuser=0&confirm=t&uuid={}&at={}"

URLs = {"freespace_nos": base_url.format("1iGG6GMj7ifRTxN9jSemnGEPI7LoL2cC7", "ca9cb465-58bb-4c0c-b4d8-0e9c5d38bed9", "APZUnTWZNWsCDlHRrAHJHgn_gF75:1721746270619"),
        "mito_nos": base_url.format("1EX0FTeXzSU4r2wrJH6yMs224vRA36Hgl", "cfbbea1b-c675-4690-8366-6288f6f3edd9", "APZUnTXZvancVIn5lbVinMumHOyF:1721745734412"),
        "mito_s": base_url.format("1H3d9MOvrGpjrrboPb4OCPyzHHsFRl27x", "9c5e33a1-d959-4e4e-afb6-401ba02b7028", "APZUnTXddzxPaJPMGyAcoTEYD08u:1721749901234"),
        "ER_nos": base_url.format("1MlEPJTaYT2G0-9XTXU2-EHsQdBMZWX13", "1a964eff-4b4a-40b3-abcc-752adcd07110", "APZUnTXCVzYJ-AmJdHGTYNtB6_G7:1721746405820"),
        "ER_s": base_url.format("15EDxXwfLJEqW93axnGCthn1THH2g9JF2", "a4ce5ee5-7be5-47db-a2eb-cd67a98cdf45", "APZUnTVGq57rAeg0ErSoJR00cCjx:1721750133835")}

#https://stackoverflow.com/a/33505522
def add_colorbar(im, aspect=20, pad_fraction=0.5, **kwargs):
    """Add a vertical color bar to an image plot."""
    divider = axes_grid1.make_axes_locatable(im.axes)
    width = axes_grid1.axes_size.AxesY(im.axes, aspect=1./aspect)
    pad = axes_grid1.axes_size.Fraction(pad_fraction, width)
    current_ax = plt.gca()
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.sca(current_ax)
    return im.axes.figure.colorbar(im, cax=cax, **kwargs)

plt.rcParams['axes.facecolor'] = (0,0,0,0)

COLOR = 'white'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR

---

**Evaluation**

Enter the parameters of your experiments in the following form:

* If all parameters are specified, Fidelity and Ambiguity scores are reported.

* If either the density or linking distance is set to **None**, an optimisation is done for this parameter.

* If both density and liking distance are set to **None**, an optimisation is done over both.

The Diffusion coefficient and acquisition time always need to be specitfied.

In [ ]:
from scipy.interpolate import interpn

#INPUTS
environment = "ER"#@param ["freespace", "mito", "ER"]
tracking = 'conventional' # @param ["conventional", "structure-aware"]
#@markdown Diffusion coefficient (µm²/s)
D = 1#@param {type:"raw"}
#@markdown Frame acquisition time (s)
DT = 0.1#@param {type:"raw"}
#@markdown Spot density (1/µm²) - use None for optimising this parameter
dens = 0.35#@param {type:"raw"}
#@markdown Maximum linking distance (µm) - use None for optimising this parameter
dist = None#@param {type:"raw"}



#PARAMETERS CONVERTION
if D is not None:
  D = float(D)
if DT is not None:
  DT = float(DT)
if dens is not None:
  dens = float(dens)
if dist is not None:
  dist = float(dist)

charac_dist = None
if D is not None and DT is not None:
  charac_dist = np.sqrt(D * DT)
  print("Characteristic length sqrt(D * DT): {:.5f} µm".format(charac_dist))

struct = "_nos"
if tracking == "structure-aware":
  struct = "_s"

ok = True
if environment == "freespace" and tracking == "structure-aware":
  print("Cannot do structure-aware tracking in freespace")
  ok = False

if ok:
  #DATA LOADING
  mat_file = wget.download(URLs[environment + struct], out="matfile.mat")
  mat = scipy.io.loadmat(mat_file)

  dist_ths = mat["dist_ths"][0]
  denss = mat["denss"][0] / 10.1621205**2
  exp_disp = (mat["exp_disp"].T)[0]

  #PARAMETERS VALIDATION
  if charac_dist != None and (charac_dist < exp_disp[0] or charac_dist > exp_disp[-1]):
    print("Error: characteristic length {} outside of simulated parameters (min: {}, max: {})".format(charac_dist, exp_disp[0], exp_disp[-1]))
    ok = False
  elif dens != None and (dens < denss[0] or dens > denss[-1]):
    print("Error: density {} outside of simulated parameters (min: {}, max: {})".format(dens, denss[0], denss[-1]))
    ok = False
  elif dist != None and (dist < dist_ths[0] or dist > dist_ths[-1]):
    print("Error: Linking distance {} outside of simulated parameters (min: {}, max: {})".format(dist, dist_ths[0], dist_ths[-1]))
    ok = False

if ok:
  if charac_dist != None and dens != None and dist != None:
    print("All parameters set, evaluating error and ambiguity:")
    cur_err = interpn((exp_disp, denss, dist_ths), mat["avg_fwd_errs"], np.array([charac_dist, dens, dist]))
    cur_ambig = interpn((exp_disp, denss, dist_ths), mat["avg_ambigs"], np.array([charac_dist, dens, dist]))

    print("Fidelity Score = {:.1f} %".format((1 - cur_err[0]) * 100))
    print("Ambiguity Score = {:.1f} %".format(cur_ambig[0] * 100))
  elif charac_dist != None and dens != None:
      print("Estimating optimal liking distance:")
      pts = []
      for d in dist_ths:
          pts.append((charac_dist, dens, d))

      cur_errs = interpn((exp_disp, denss, dist_ths), mat["avg_fwd_errs"], pts)
      cur_ambigs = interpn((exp_disp, denss, dist_ths), mat["avg_ambigs"], pts)
      best_idx = np.argmin(cur_errs)
      print("Distance = {:.1f} µm".format(dist_ths[best_idx]))
      print("Fidelity Score = {:.1f} %".format((1 - cur_errs[best_idx]) * 100))
      print("Ambiguity Score = {:.1f} %".format(cur_ambigs[best_idx] * 100))
      plt.figure(facecolor=[0.219607843,0.219607843,0.219607843])
      plt.plot(dist_ths, (1 - cur_errs)*100)
      plt.plot(dist_ths[best_idx], (1 - cur_errs[best_idx])*100, '*r')
      plt.ylim([0, 100])
      plt.xlabel('Linking distance (µm)')
      plt.ylabel('Percentage of correct displacements')
  elif charac_dist != None and dist != None:
      print("Estimating optimal density:")
      pts = []
      for d in denss:
          pts.append((charac_dist, d, dist))

      cur_errs = interpn((exp_disp, denss, dist_ths), mat["avg_fwd_errs"], pts)
      cur_ambigs = interpn((exp_disp, denss, dist_ths), mat["avg_ambigs"], pts)
      best_idx = np.argmin(cur_errs)
      print("density = {:.5f} spot/µm²".format(denss[best_idx]))
      print("Fidelity Score = {:.1f} %".format((1 - cur_errs[best_idx]) * 100))
      print("Ambiguity Score = {:.1f} %".format(cur_ambigs[best_idx] * 100))
      plt.figure(facecolor=[0.219607843,0.219607843,0.219607843])
      plt.plot(denss, (1 - cur_errs)*100)
      plt.plot(denss[best_idx], (1 - cur_errs[best_idx])*100, '*r')
      plt.ylim([0, 100])
      plt.xlabel('Density (1/µm²)')
      plt.ylabel('Percentage of correct displacements')
  elif charac_dist != None :
      print("Estimating optimal density and distances:")
      pts = []
      for de in denss:
          for di in dist_ths:
              pts.append((charac_dist, de, di))

      cur_errs = interpn((exp_disp, denss, dist_ths), mat["avg_fwd_errs"], pts)
      cur_errs = cur_errs.reshape(len(denss), len(dist_ths))
      cur_ambigs = interpn((exp_disp, denss, dist_ths), mat["avg_ambigs"], pts)
      cur_ambigs = cur_ambigs.reshape(len(denss), len(dist_ths))

      best_idx = np.argmin(cur_errs)
      [i,j] = np.unravel_index(best_idx, cur_errs.shape)
      print("Density = {:.5f} spot/µm²".format(denss[i]))
      print("Distance = {:.1f} µm".format(dist_ths[j]))
      print("Fidelity Score = {:.1f} %".format((1 - cur_errs[i,j]) * 100))
      print("Ambiguity Score = {:.1f} %".format(cur_ambigs[i,j] * 100))
      plt.figure(figsize=(20,20), facecolor=[0.219607843,0.219607843,0.219607843])
      im = plt.imshow((1 - cur_errs) * 100, vmin=0, vmax=100, origin="lower", extent=(dist_ths[0], dist_ths[-1], denss[0], denss[-1]), cmap="plasma")
      cb = add_colorbar(im)
      plt.plot(dist_ths[j], denss[i], '*r')
      plt.xlabel('Linking distance (µm)')
      plt.ylabel('Density (1/µm²)')
      cb.ax.set_ylabel("% correct disps", rotation=270, labelpad=10)
  else:
      print("ERROR: Unrecognised optimisation")